In [1]:
!pip install --q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.0 MB/s eta 0:00:00


In [20]:
!pip install --q evaluate jiwer datasets[audio] librosa langchain langchain-community langchain-core langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.8 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
import torch, os

HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [5]:
from datasets import load_dataset

dataset = load_dataset("lmms-lab/librispeech", "librispeech_test_clean", revision='0ea039d2ced2e10e9961db299c5b9ef31034cbd7', split='librispeech_test_clean[0:150]')

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

(…)speech_test_clean-00000-of-00002.parquet:   0%|          | 0.00/312M [00:00<?, ?B/s]

(…)speech_test_clean-00001-of-00002.parquet:   0%|          | 0.00/301M [00:00<?, ?B/s]

Generating librispeech_test_clean split:   0%|          | 0/2620 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['audio', 'gt', 'prompt', 'source', 'task'],
    num_rows: 150
})

In [55]:
# sample
from IPython.display import Audio

sample = dataset[1]
audio_sample = sample['audio']
audio_transcription = sample['gt']

original_sr = audio_sample['sampling_rate']
original_sr

192000

In [56]:
import librosa

target_sr = 16000
audio_sample['array'] = librosa.resample(audio_sample['array'], orig_sr=original_sr, target_sr=target_sr)
audio_sample['sampling_rate'] = target_sr

Audio(audio_sample['array'], rate=audio_sample['sampling_rate'])

In [9]:
audio_transcription

'HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE'

In [10]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en").to(DEVICE)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

In [11]:
audio_sample['sampling_rate']

16000

In [12]:
input_features = processor(
    audio_sample["array"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt"
).input_features
input_features = input_features.to(DEVICE)
input_features

tensor([[[ 0.1188,  0.0630,  0.0184,  ..., -0.6991, -0.6991, -0.6991],
         [-0.1339,  0.0589,  0.0182,  ..., -0.6991, -0.6991, -0.6991],
         [-0.2594, -0.0884, -0.1486,  ..., -0.6991, -0.6991, -0.6991],
         ...,
         [-0.6991, -0.6991, -0.6991,  ..., -0.6991, -0.6991, -0.6991],
         [-0.6991, -0.6991, -0.6991,  ..., -0.6991, -0.6991, -0.6991],
         [-0.6991, -0.6991, -0.6991,  ..., -0.6991, -0.6991, -0.6991]]],
       device='cuda:0')

In [13]:
predicted_ids = model.generate(input_features, do_sample=True, temperature=1.0)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription[0].strip()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat-mutton pieces to be ladled out in thick, peppered, flower-fattened sauce.'

# Pipeline

In [98]:
# mainly based on the real paper
# i only modify it a little bit so that it performs better
PROMPT = """
### Instruction
You are an ASR transcript selector. You have a few transcripts generated by an automatic speech
recognition model. Your task is to generate the most likely transcript from them. If the generated
transcripts have grammatical or logical errors, you will modify them accordingly to produce the
most accurate and coherent transcript. Directly return the transcript without explanation.

### Input
{transcripts}

### Response
"""

In [102]:
import re, random
from langchain_huggingface import ChatHuggingFace
from langchain import HuggingFaceHub

def normalize_text(text):
  text = re.sub(r'-', ' ', text)
  text = re.sub(r'[^\w\s]', '', text).lower()
  return text

class WhisperingLlamaSimplified:
  def __init__(self, model, processor, llm_model, prompt):
    self.model = model
    self.processor = processor
    model_kwargs = {
      "device": False,
      "max_new_tokens": 200,
      "return_full_text": False,
    }
    llm = HuggingFaceHub(repo_id=llm_model, model_kwargs=model_kwargs)
    self.chat_model = ChatHuggingFace(llm=llm)
    self.prompt = prompt

  def __resample_audio(self, sample, target_sr):
    original_sr = sample['sampling_rate']
    sample['array'] = librosa.resample(sample['array'], orig_sr=original_sr, target_sr=target_sr)
    sample['sampling_rate'] = target_sr
    return sample

  def __generate_one_transcription(self, audio, temperature):
    audio = self.__resample_audio(audio, 16000)
    input_features = self.processor(
        audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt"
    ).input_features.to(DEVICE)
    predicted_ids = self.model.generate(input_features, do_sample=True, temperature=temperature)
    transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)
    return transcription[0].strip()

  def __generate_k_transcriptions(self, audio, k):
    audio = self.__resample_audio(audio, 16000)
    random_options = [i/100 for i in range(70, 101)]
    transcriptions = []
    for i in range(k):
      temperature = random.choice(random_options)
      raw_transcription = self.__generate_one_transcription(audio, temperature)
      normalized_transcription = normalize_text(raw_transcription)
      transcriptions.append(normalized_transcription)
    return transcriptions

  def generate_best_transcription(self, audio, k):
    transcriptions = self.__generate_k_transcriptions(audio, k)
    transcriptions = '\n'.join(transcriptions)
    prompt = self.prompt.format(transcripts=transcriptions)
    response = self.chat_model.invoke(prompt).content
    return normalize_text(response)

In [103]:
wlsimplified = WhisperingLlamaSimplified(model, processor, "meta-llama/Meta-Llama-3-8B-Instruct", PROMPT)

In [75]:
from evaluate import load
wer = load("wer")

In [104]:
from tqdm import tqdm

transcriptions = []
ground_truths = []
wer_values = []

for row in tqdm(dataset):
  audio = row['audio']
  gt = normalize_text(row['gt'])
  transcription = wlsimplified.generate_best_transcription(audio, 5)
  print(transcription)
  transcriptions.append(transcription)
  ground_truths.append(gt)
  wer_values.append(wer.compute(predictions=[transcription], references=[gt]))

  1%|          | 1/150 [00:01<03:58,  1.60s/it]

 instruction


  1%|▏         | 2/150 [00:02<02:57,  1.20s/it]

stuffed into his belly he counselled him


  2%|▏         | 3/150 [00:04<03:16,  1.34s/it]

...

youre lazy judge why when you hear because the thing had been such a scare he continued to fix me


 99%|█████████▉| 149/150 [05:02<00:01,  1.72s/it]

 instruction


100%|██████████| 150/150 [05:07<00:00,  2.05s/it]

 instruction
he quitted the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire and the fire


In [105]:
avg_wer = sum(wer_values)/len(wer_values)
avg_wer

1.1133239526953995